### 0.补充
在介绍高斯混合模型的变分推断之前先补充两个分布的知识，一个是高斯-Wishart分布，它是高维高斯分布的共轭先验，另一个是学生t分布，它是高斯-Gamma分布对精度积分的边缘概率分布，下面分别介绍

#### 高斯-Wishart分布

对于$D$维向量$x$的多元高斯分布$N(x\mid\mu,\Lambda^{-1})$，假设精度$\Lambda$已知，则均值$\mu$的共轭先验分布仍为高斯分布，但对于均值已知，精度未知的情况，其共轭先验是Wishart分布，定义如下：   

$$
\mathcal{W}(\Lambda\mid W,\mathcal{v})=B|\Lambda|^{\frac{\mathcal{v}-D-1}{2}}exp(-\frac{1}{2}Tr(W^{-1}\Lambda))
$$  

其中，$\mathcal{v}$被称为分布的自由度数量，$W$是一个$D\times D$的标量矩阵，$Tr(\cdot)$表示矩阵的迹，归一化系数$B$为：  

$$
B(W,\mathcal{v})=|W|^{-\frac{\mathcal{v}}{2}}(2^{\frac{\mathcal {v}D}{2}}\pi^{\frac{D(D-1)}{4}}\prod_{i=1}^D\Gamma(\frac{\mathcal{v}+1-i}{2}))^{-1}
$$  

而对于均值和精度均未知的情况，那么类似于一元变量的推理方法，可以得到其共轭先验为：   

$$
p(\mu,\Lambda\mid\mu_0,beta,W,\mathcal{v})=N(\mu\mid\mu_0,(\beta\Lambda)^{-1})\mathcal{W}(\Lambda\mid W,\mathcal{v})
$$  

该分布便是高斯-Wishart分布，或者正态-Wishart分布

#### 学生t分布
根据之前的推导我们知道，对于$\mu$已知，$\tau$未知的一元高斯分布$N(x\mid\mu,\tau^{-1})$，它的共轭分布为Gamma分布，不妨表示为$Gam(\tau\mid a,b)$，如果把精度积分掉，那么就得到了一个边缘概率分布：   

$$
p(x\mid \mu,a,b)=\int_0^\infty N(x\mid\mu,\tau^{-1})Gam(\tau\mid a,b)d\tau\\
=\int_0^\infty \frac{b^ae^{(-b\tau)}\tau^{a-1}}{\Gamma(a)}(\frac{\tau}{2\pi})^{\frac{1}{2}}exp(-\frac{\tau}{2}(x-\mu)^2)d\tau\\
=\frac{b^a}{\Gamma(a)}(\frac{1}{2\pi})^{\frac{1}{2}}\int_0^\infty(\frac{z}{b+\frac{(x-\mu)^2}{2}})^{a-\frac{1}{2}}e^{-z}[b+\frac{(x-\mu)^2}{2}]^{-1}dz（令z=\tau(b+\frac{(x-\mu)^2}{2})）\\
=\frac{b^a}{\Gamma(a)}(\frac{1}{2\pi})^{\frac{1}{2}}(b+\frac{(x-\mu)^2}{2})^{-a-\frac{1}{2}}\int_0^\infty z^{(a+\frac{1}{2})-1}e^{-z}dz\\
=\frac{b^a}{\Gamma(a)}(\frac{1}{2\pi})^{\frac{1}{2}}(b+\frac{(x-\mu)^2}{2})^{-a-\frac{1}{2}}\Gamma(a+\frac{1}{2})
$$  

我们对参数做一下调整，令$\mathcal{v}=2a,\lambda=\frac{a}{b}$，那么这就是学生t分布：   

$$
St(x\mid\mu,\lambda,\mathcal{v})=\frac{\Gamma(\frac{\mathcal{v}}{2}+\frac{1}{2})}{\Gamma(\frac{\mathcal{v}}{2})}(\frac{\lambda}{\pi\mathcal{v}})^{\frac{1}{2}}(1+\frac{\lambda(x-\mu)^2}{\mathcal{v}})^{-\frac{\mathcal{v}}{2}-\frac{1}{2}}
$$  

这里，参数$\lambda$被称为t分布的精度（注意：通常情况下，它不是方差的倒数），参数$\mathcal{v}$被称为自由度，它的作用如下图所示（其中$\mu=0,\lambda=1$），对于$\mathcal{v}=1$的情况下，t分布是柯西分布，而对于$\mathcal{v}\rightarrow \infty$的情况下，t分布$St(x\mid\mu,\lambda,\mathcal{v})$就变成了高斯分布$N(x\mid\mu,\lambda^{-1})$   


![avatar](./source/15_t分布1.png)

需要注意的一点是，学生t分布可以看作无限个同均值不同精度的高斯分布相加所得到（高斯混合模型的极限），所以t分布通常有更长的“尾巴”，建模时具有更强的鲁棒性，如下图是用t分布（红线）和高斯分布（绿线）建模的对比，在未引入噪声数据之前，如（a）图，t分布和高斯分布的建模效果相同，而在引入噪声数据之后，如（b）图，会对高斯分布模型造成较大的影响，而t分布几乎不受影响

![avatar](./source/15_t分布2.png)  


高维学生t分布：   

将一维学生t分布扩展到高维，可以写作  

$$
St(x\mid\mu,\Lambda,\mathcal{v})=\frac{\Gamma(\frac{D}{2}+\frac{\mathcal{v}}{2})}{\Gamma(\frac{\mathcal{v}}{2})}\frac{|\Lambda|^{\frac{1}{2}}}{(\pi\mathcal{v})^{\frac{D}{2}}}[1+\frac{(x-\mu)^T\Lambda(x-\mu)}{\mathcal{v}}]^{-\frac{D}{2}-\frac{\mathcal{v}}{2}}
$$

其中，$D$是$x$的维度，另外t分布具有如下的性质：   

（1）$E[x]=\mu,\mathcal{v}>1$；   

（2）$cov[x]=\frac{\mathcal{v}}{\mathcal{v}-2}\Lambda^{-1},\mathcal{v}>2$；   

（3）$mode[x]=\mu$